In [1]:
import pandas as pd
import re
import glob
import os
from scipy import stats
import numpy as np
from scipy.optimize import minimize 
from scipy.optimize import newton

In [2]:
d = {'Cys': 'C', 'Asp': 'D', 'Ser': 'S', 'Gln': 'Q', 'Lys': 'K',
     'Ile': 'I', 'Pro': 'P', 'Thr': 'T', 'Phe': 'F', 'Asn': 'N', 
     'Gly': 'G', 'His': 'H', 'Leu': 'L', 'Arg': 'R', 'Trp': 'W', 
     'Ala': 'A', 'Val':'V', 'Glu': 'E', 'Tyr': 'Y', 'Met': 'M'}
d2=dict((v, k) for k, v in d.items())

In [3]:
def truncated_mean(lam):
    mu=(1/lam-np.exp(-lam)*(1+1/lam))/(1-np.exp(-lam))
    return mu
def equation(lam, given_mu):
    return truncated_mean(lam) - given_mu

def truncated_expo_density(x, lmbda):
    return lmbda * np.exp(-lmbda * x) / (1 - np.exp(-lmbda))

def truncated_expo_cdf(x, lmbda):
    return (1 - np.exp(-lmbda * x)) / (1 - np.exp(-lmbda))

In [4]:
#read in all deep mutational files
path = '../Data/ProteinGym_substitutions/'
DMS_files=glob.glob(os.path.join(path, '*.csv'))

all_dms=[]
i=0
for f in DMS_files:
    dms=pd.read_csv(f)
    dms['mut_type']=dms['mutant'].str.replace(r'\d+', '', regex=True)
    dms['mut_type_len']=dms['mut_type'].str.len()
    dms=dms[dms['mut_type_len']==2]
    dms['fit_rank_all']=dms['DMS_score'].rank(pct=True)
    all_dms.append(dms)
    i=i+1
all_dms=pd.concat(all_dms)

In [5]:
#pre-process data frame
AA_list=['Arg','Lys','Gln','Glu','Asp',
         'Asn','His','Ser','Thr','Ala',
         'Val','Ile','Leu','Met','Pro',
         'Gly','Tyr','Phe','Trp','Cys']
aa_list=[d[a] for a in AA_list]

In [6]:
all_dms['AA1']=all_dms['mut_type'].str[0]
all_dms['AA2']=all_dms['mut_type'].str[1]

In [7]:
aa_pair=[]
lmbda_list=[]
for i in range(20):
    for j in range(20):
        if i!=j:
            data = all_dms[(all_dms['AA1']==aa_list[i])&(all_dms['AA2']==aa_list[j])]
            given_mu=np.mean(data.fit_rank_all.tolist())
            # Extract the lambda parameter that maximizes the likelihood
            lmbda_opt = newton(equation, 1, args=(given_mu,))
            lmbda_list.append(lmbda_opt)
            aa_pair.append(aa_list[i]+'-'+aa_list[j])
            

In [8]:
df_exchangeability=pd.DataFrame({"AA_pair":aa_pair,
                                 "lambda":lmbda_list})

In [10]:
df_exchangeability.to_csv('../exchangeability_table.csv',index=False)